In [ ]:
#!pip install streamlit
#pip install imbalanced-learn

In [1]:
# import the functions:

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#import functions
import re
import nltk
import joblib
import pandas as pd
from gensim.models import Word2Vec

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\calum\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\calum\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Load the SavedModel
#model = tf.saved_model.load("C:/Users/Juliane/Documents/GitHub/labs_repo/Final project/tf0_model")
model_path = r"C:\Users\calum\Desktop\Ironhack\Week8\Final Project\tf2_model\saved_model.pb"
model = tf.keras.models.load_model(model_path)

# Load the tokenizer
tokenizer = joblib.load(r"C:\Users\calum\Desktop\Ironhack\Week8\Final Project\saved_tokenizer.joblib")

# Load pre-trained Word2Vec model
word2vec_model = Word2Vec.load(r"C:\Users\calum\Desktop\Ironhack\Week8\Final Project\word2vec.model")

OSError: Unable to open file (file signature not found)

In [ ]:
#model_path = "C:/Users/Juliane/Documents/GitHub/labs_repo/Final project/Final_streamlit/tf2_model"
#loaded_model = tf.keras.models.load_model(model_path)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.MULTILINE)
    words = [word for word in text.split() if word.isalnum() and word.isalpha()]
    cleaned_text = ' '.join(words)
    return cleaned_text

def tokenizing_and_stopwords(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    text = " ".join(filtered_tokens)
    return text

def lemmatization_and_stopwords(text):
    if not text:
        return ""
    tokens = word_tokenize(text)
    clean_text = []
    lemmatizer = nltk.WordNetLemmatizer()
    for token in tokens:
        if token.lower() not in stopwords.words('english') and len(token) > 3:
            token = lemmatizer.lemmatize(token)
            clean_text.append(token)
    result_text = " ".join(clean_text)
    return result_text


In [ ]:
import streamlit as st

# Placeholder for loading the trained model and vectorizer
#def load_model_and_vectorizer():    

def classify_message(message):
    # Preprocess the input message
    cleaned_message = clean_text(message)
    tokenized_message = tokenizing_and_stopwords(cleaned_message)
    lemmatized_message = lemmatization_and_stopwords(tokenized_message)

    # Convert text to sequence
    sequence = tokenizer.texts_to_sequences(lemmatized_message)

    # Pad sequence
    padded_sequence = pad_sequences(sequence, maxlen=500)  # Assuming the max_sequence_length is 500

    # Make predictions using the loaded model
    prediction = model.predict(padded_sequence)
    
    return prediction[0]


In [ ]:
def main():

    st.title("Detect Cyberbullying")

    # User input for message
    message = st.text_area("Enter a message:")

    if st.button("Bullying or not?"):
        
        if message: 
            # Classify the message
            result = classify_message(message)

            # Display the result
            if result > 0.5:
                st.error("This is bullying!")
            else:
                st.success("No bullying content.")
        
        else:
            st.warning("Please enter a message to classify.")

    #st.image("cb.png", use_column_width=True)
    

if __name__ == "__main__":
    main()

In [ ]:
# Test the classify_message function with a sample message
sample_message = "you bitch"
result = classify_message(sample_message)

# Display the result
result